# 不等式のQUBO変換を用いて基底エネルギー以下のサンプルを得る

### import

In [93]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random

### 各変数

In [94]:
# X
matrix = np.random.randn(3,5)
X= pd.DataFrame(matrix, columns=list('ABCDE'))
X

,A,B,C,D,E
0,-0.259570,0.326301,0.281129,0.525707,0.240735
1,1.533296,0.331656,-1.470536,2.230062,0.509792
2,-1.308316,-0.053805,-1.185971,-0.352907,-1.408064


In [95]:
# y
y = pd.Series([1, 0, 1])
y

0    1
1    0
2    1
dtype: int64

In [96]:
selected_col_num = 1

In [97]:
max_X_element = X.max(axis=1).max() 
min_X_element = X.min(axis=1).min() 

In [98]:
num_samples = X.shape[0]
num_cols = X.shape[1]

## 特徴量選択
 - select_cols, else_colsを決める

In [99]:
selector = SelectKBest(score_func=f_regression, k=selected_col_num) 
selector.fit(X, y)
mask = selector.get_support() 

In [100]:
select_cols = []
else_cols = []
for ii in range(len(mask)):
    if mask[ii] == 0:
        else_cols.append(X.columns[ii])
    else:
         select_cols.append(X.columns[ii])

select_cols

['D']

## すべての不等式が成り立つ：条件
したいこと→ハミルトニアンを最小化させる（xを最適化）

In [101]:
# W : ay - byの最大値

#xの要素で最大値とxの要素の最小値を知れば推定はできる
W = int((max_X_element *1*num_samples - min_X_element*1*num_samples)//1)

In [102]:
def make_y(y_before):
    select_index = random.randrange(num_samples)
    y_before[select_index] = 1 - y_before[select_index] 
    y_after = y_before
    return y_after

In [103]:
def make_ax_by(y_after):
    ax_by = []
    for i in range(selected_col_num):
        for j in range(num_cols - selected_col_num):
            ax_by.append(np.dot(X[select_cols[i]], y_after) - np.dot(X[else_cols[j]], y_after) +1)
    return ax_by 

In [104]:
def H(ax_by, index_ax_by):
    z = [0]*W
        
    each_ax_by = ax_by[index_ax_by]
    int_each_axby = each_ax_by //1
    sum_w_z = 0
    
    if int_each_axby >= 1:
        z[int(int_each_axby)] = 1
        sum_w_z = sum([int_each_axby*z[w] for w in range(W)])
    else:
        z[0] = 1
        sum_w_z = sum([1*z[w] for w in range(W)])
        
    sum_z = sum(z)
    z_series = pd.Series(z)

    ans_h =  (1-sum_z)**2 + (sum_w_z -  int(int_each_axby))**2
    return ans_h,  z_series

In [105]:
#know ori_H_sum
ax_by = make_ax_by(y)

ori_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
ori_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
ori_z_df = pd.concat(ori_z_list, axis = 1)

print(ori_H_sum)
print([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
print(ori_z_df)

1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   0  0  0  0
2   1  0  1  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0


In [106]:
#find ans_y which H is as low as ori_y

y_after =  make_y(y)
ax_by = make_ax_by(y_after)

each_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
each_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
each_z_df = pd.concat(each_z_list, axis = 1)


if each_H_sum <= ori_H_sum:
    print(each_H_sum)
    print([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
    print(each_z_df)
    print(y_after) 

0.0
[0.0, 0.0, 0.0, 0.0]
    0  1  2  3
0   0  0  0  0
1   0  0  0  0
2   0  1  0  0
3   1  0  0  0
4   0  0  0  1
5   0  0  1  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    1
2    1
dtype: int64


In [107]:
#少なめloop
iii = 0
for ii in range(100):
    y_after =  make_y(y)
    ax_by = make_ax_by(y_after)

    a_H_sum =  sum([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))])
    a_z_list =  [H(ax_by, index_ax_by)[1] for index_ax_by in range(len(ax_by))]
    a_z_df = pd.concat(a_z_list, axis = 1)
    if a_H_sum <= ori_H_sum:
        iii += 1
        print(a_H_sum) #0.0がほとんど
        print([H(ax_by, index_ax_by)[0] for index_ax_by in range(len(ax_by))]) #よって[0.0, 0.0, 0.0, 0.0]
        print(a_z_df)
        print(y_after) 

1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   0  0  0  0
2   1  0  1  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    0
2    1
dtype: int64
0.0
[0.0, 0.0, 0.0, 0.0]
    0  1  2  3
0   0  0  0  0
1   0  0  0  0
2   0  1  0  0
3   1  0  0  0
4   0  0  0  1
5   0  0  1  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    1
2    1
dtype: int64
0.0
[0.0, 0.0, 0.0, 0.0]
    0  1  2  3
0   0  0  0  0
1   0  0  0  0
2   1  1  0  0
3   0  0  0  1
4   0  0  0  0
5   0  0  1  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    0
1    1
2    1
dtype: int64
0.0
[0.0, 0.0, 0.0, 0.0]
    0  1  2  3
0   0  0  0  0
1   0  0  0  0
2   0  1  0  0
3   1  0  0  0
4   0  0  0  1
5   0  0  1  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    1
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1

1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   1  0  1  0
2   0  0  0  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    0
1    0
2    1
dtype: int64
0.0
[0.0, 0.0, 0.0, 0.0]
    0  1  2  3
0   0  0  0  0
1   0  0  0  0
2   1  1  0  0
3   0  0  0  1
4   0  0  0  0
5   0  0  1  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    0
1    1
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   1  0  1  0
2   0  0  0  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    0
1    0
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   0  0  0  0
2   1  0  1  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    0
2    1
dtype: int64
0.0
[0.0, 0.0, 0.0, 0.0]
    0  1  2  3
0   0  0  

    0  1  2  3
0   0  1  0  0
1   0  0  0  0
2   1  0  1  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    0
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   1  0  1  0
2   0  0  0  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    0
1    0
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   0  0  0  0
2   1  0  1  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    1
1    0
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   1  0  1  0
2   0  0  0  1
3   0  0  0  0
4   0  0  0  0
5   0  0  0  0
6   0  0  0  0
7   0  0  0  0
8   0  0  0  0
9   0  0  0  0
10  0  0  0  0
0    0
1    0
2    1
dtype: int64
1.0
[0.0, 1, 0.0, 0.0]
    0  1  2  3
0   0  1  0  0
1   0  0  0  0
2   1  

## py-sat

In [ ]:
from pysat.formula import CNF
from pysat.examples.lbx import LBX

In [ ]:
formula = CNF(from_file='input.cnf')
mcsls = LBX(formula)
for mcs in mcsls.enumerate():
    print(mcs)

# d-wave 
-[What is Quantum Annealing?](https://docs.dwavesys.com/docs/latest/c_gs_2.html)

## neal usage
- [dwave-neal](https://docs.ocean.dwavesys.com/projects/neal/en/latest/)

In [ ]:
#!pip install dwave-neal

In [139]:
import neal

sampler = neal.SimulatedAnnealingSampler()

h = {0: -1, 1: -1}
J = {(0, 1): -1}
response = sampler.sample_ising(h, J)
response

SampleSet(rec.array([([1, 1], -3., 1)],
          dtype=[('sample', 'i1', (2,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1], {'beta_range': [0.34657359027997264, 4.605170185988092], 'beta_schedule_type': 'geometric'}, 'SPIN')

- [dwave-neal→Simulated Annealing Sampler](https://dwave-neal-docs.readthedocs.io/en/latest/reference/sampler.html)

In [ ]:
#classSimulatedAnnealingSampler

sampler = neal.SimulatedAnnealingSampler()
h = {'a': 0.0, 'b': 0.0, 'c': 0.0}
J = {('a', 'b'): 1.0, ('b', 'c'): 1.0, ('a', 'c'): 1.0}

response = sampler.sample_ising(h, J)

for sample in response:  # doctest: +SKIP
    print(sample)

## dimod usage
- [dimod](https://docs.ocean.dwavesys.com/projects/dimod/en/latest/)
- [Ising, QUBO, and BQMs](https://docs.ocean.dwavesys.com/projects/dimod/en/latest/reference/bqm/binary_quadratic_model.html)

$$ H(x\in \{0,1\}^3) = x_1 + x_2 + 2x_1 * x_2 - x_2 * x_3$$


- 北井さんのコードは適用できない

In [ ]:
#!pip freeze #dimod==0.9.3

In [ ]:
import dimod

In [ ]:
#lin = [1, 1, 0]
#quad = {(0, 1):2, (1, 2):-1}
#bqm = dimod.BinaryQuadraticModel(lin, quad)

In [ ]:
#bqm.to_ising()

In [ ]:
#bqm = dimod.BinaryQuadraticModel({'a': 0.0, 'b': -1.0, 'c': 0.5},　{('a', 'b'): -1.0, ('b', 'c'): 1.5},　1.4,　dimod.SPIN)
#bqm.to_qubo()

## dimod.BinaryQuadraticModel

In [ ]:
bqm = dimod.BinaryQuadraticModel({0: -1, 1: -1}, {(0, 1): 2}, 0.0, dimod.BINARY)
bqm_ising = bqm.change_vartype(dimod.SPIN, inplace=False)
bqm_ising 

In [ ]:
h = {0: 0.0, 1: 0.0}
J = {(0, 1): -1.0}
bqm = dimod.BinaryQuadraticModel.from_ising(h, J)
response = dimod.ExactSolver().sample(bqm)
for sample, energy in response.data(['sample', 'energy']): 
    print(sample, energy)

In [ ]:
h = {'a': -0.5, 'b': 1.0}
J = {('a', 'b'): -1.5}
sampleset = dimod.ExactSolver().sample_ising(h, J)
print(sampleset)

In [ ]:
Q = {('a', 'b'): 2.0, ('a', 'a'): 1.0, ('b', 'b'): -0.5}
sampleset = dimod.ExactSolver().sample_qubo(Q)
sampleset

In [ ]:
bqm = dimod.BinaryQuadraticModel({'a': 1.5}, {('a', 'b'): -1}, 0.0, 'SPIN')
sampleset = dimod.ExactSolver().sample(bqm)
sampleset

In [ ]:
bqm = dimod.BinaryQuadraticModel.from_qubo({('a', 'b'): 1})
sampler = dimod.NullSampler()
sampleset = sampler.sample(bqm)
len(sampleset)

In [ ]:
bqm = dimod.BinaryQuadraticModel.from_qubo({('a', 'b'): 1})
sampler = dimod.NullSampler(parameters=['a'])
sampleset = sampler.sample(bqm, a=5)
sampleset 

In [ ]:
h = {'a': -0.5, 'b': 1.0}
J = {('a', 'b'): -1.5}
sampleset = dimod.SimulatedAnnealingSampler().sample_ising(h, J)
sampleset